# [Day 11] 로봇을 위한 수학 1: 3차원의 세계로

4주차에 오신 것을 환영합니다! 이제부터는 로봇 공학의 꽃, **수학**과 **하드웨어**를 다룹니다.

지난주까지 우리는 화면(2D)에서 손을 찾았습니다. 하지만 로봇 손은 입체적(3D)으로 움직이죠.
다행히 MediaPipe는 우리에게 3차원 좌표(`x`, `y`, `z`)를 모두 알려줍니다.
오늘은 이 `z`축(깊이)이 무엇인지 직접 눈으로 확인해보겠습니다.

## 1. 3D 좌표계 이해하기

- **x**: 화면의 가로 (왼쪽 $\leftrightarrow$ 오른쪽)
- **y**: 화면의 세로 (위 $\leftrightarrow$ 아래)
- **z**: **깊이 (카메라 $\leftrightarrow$ 내 몸)**
    - 카메라에 가까워질수록 z값은 어떻게 변할까요?
    - 카메라에서 멀어질수록 z값은 어떻게 변할까요?

In [ ]:
import cv2 as cv
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1) # 헷갈리니까 손은 하나만
mp_drawing = mp.solutions.drawing_utils

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv.flip(frame, 1)
    image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # 검지 끝(8번)의 3D 좌표 가져오기
            idx_tip = hand_landmarks.landmark[8]
            
            # 좌표 출력 (소수점 둘째자리까지만)
            text = f"X:{idx_tip.x:.2f} Y:{idx_tip.y:.2f} Z:{idx_tip.z:.2f}"
            
            # 화면에 글씨 쓰기
            cv.putText(frame, text, (10, 50), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv.imshow('3D Check', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

### [실험] 손을 앞뒤로 움직여보세요

위 코드를 실행하고 손을 카메라 쪽으로 가까이 가져갔다가, 멀리 떨어뜨려 보세요.
**Z값이 어떻게 변하나요?**

MediaPipe에서 `z`값은 손목(0번)을 기준으로 상대적인 깊이를 나타냅니다.
값이 작아질수록(음수가 될수록) 카메라에 가깝다는 뜻일까요? 직접 확인해보는 것이 숙제입니다!

## 2. Q&A

**Q: x, y 좌표가 0.5 같은 소수점인데 픽셀 위치가 맞나요?**  
A: 아닙니다! 저번 시간에 배웠듯 이것은 '비율'입니다. 실제 픽셀 위치를 알기 위해서는 화면 크기(가로, 세로)를 곱해줘야 합니다.

**Q: 손이 없는데 자꾸 손이라고 인식해요.**  
A: 배경에 살색과 비슷한 물체가 있거나 빛 반사가 심하면 오작동할 수 있습니다. `min_detection_confidence` 값을 0.7이나 0.8로 높여보세요.

내일은 이 점 3개를 이어서 **'벡터(Vector)'**를 만들고, 손가락이 굽혀진 **'각도(Angle)'**를 구해보겠습니다.